<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB29_Chain_of_Thought_Prompting_en_action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB29 : Chain-of-Thought Prompting en action

**Objectif:** Utiliser différents styles de prompting Chain-of-Thought (CoT) pour voir comment les LLMs peuvent décomposer les tâches de raisonnement en étapes avant de répondre.

**Durée estimée:** 10–15 minutes

**Livrable:** Notebook permettant de comparer les réponses avec et sans CoT, à travers des tâches mathématiques, logiques et de bon sens.

---

## Step 1: Setup

Installation des dépendances nécessaires.

In [1]:
# Installation des packages
!pip install -q openai python-dotenv

### Configuration de la clé API OpenAI

**Sur Google Colab**, vous pouvez configurer votre clé API de deux façons:

#### Option 1: Utiliser les Secrets de Colab (Recommandé)
1. Cliquez sur l'icône 🔑 (clé) dans la barre latérale gauche
2. Ajoutez un nouveau secret nommé `OPENAI_API_KEY`
3. Collez votre clé API
4. Activez l'accès au notebook

#### Option 2: Définir directement (moins sécurisé)
Décommentez et modifiez la ligne ci-dessous avec votre clé API.

In [2]:
import os

# Option 1: Utiliser les Secrets de Colab (recommandé)
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ Clé API chargée depuis les Secrets Colab")
except:
    # Option 2: Définir manuellement (décommentez et ajoutez votre clé)
    # os.environ['OPENAI_API_KEY'] = "sk-votre-cle-api-ici"
    print("⚠️ Veuillez configurer votre clé API OpenAI")
    print("   Utilisez les Secrets Colab ou décommentez la ligne ci-dessus")

✅ Clé API chargée depuis les Secrets Colab


## Step 2: Fonction Helper

Création d'une fonction pour interagir avec l'API OpenAI.

In [4]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def ask(prompt, show=True):
    """
    Envoie un prompt à GPT-4o-mini et retourne la réponse.

    Args:
        prompt: Le texte du prompt
        show: Si True, affiche la réponse

    Returns:
        La réponse du modèle
    """
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}]
    )
    ans = resp.choices[0].message.content
    if show:
        print(ans)
    return ans

# Test 1 de la fonction
print("🧪 Test de la fonction ask():")
ask("Dis bonjour en une phrase courte.")

# Test 2 de la fonction
print("🧪 Test de la fonction ask():")
ask("Dis bonjour en une phrase longue.")

🧪 Test de la fonction ask():
Bonjour !
🧪 Test de la fonction ask():
Bonjour, cher interlocuteur, j'espère que cette salutation chaleureuse que je t'adresse en ce moment t'apporte un peu de joie et de lumière dans ta journée.


"Bonjour, cher interlocuteur, j'espère que cette salutation chaleureuse que je t'adresse en ce moment t'apporte un peu de joie et de lumière dans ta journée."

## Step 3: Baseline vs CoT — Math

Comparaison entre une approche baseline (sans guidage) et avec Chain-of-Thought sur un problème mathématique.

In [26]:
# q = "A farmer has 17 sheep. All but 9 run away. How many are left?"
q = "Un fermier possède 17 moutons. Tous sauf 9 s'enfuient. Combien en reste-t-il ?"
print("\n" + "="*60)
print("📊 Problème mathématique: Comptage de moutons")
print("="*60)
print(f"Question: {q}\n")

print("_"*55+"\n")
print("1°) Basique (sans guidage) :")
print("_"*55+"\n")
baseline_math = ask(q)


print("\n"*2 + "_"*55)
print("\n2°) Avec chaîne de pensée ou CoT (Chain-of-Thought) :")
print("_"*55+"\n")
#cot_math = ask(q + "\n\nThink step by step before giving the final answer.")
cot_math = ask(q + "\n\nRéfléchis étape par étape avant de donner la réponse finale.")

print("\n"*2 + "="*60)
print("💡 Observation:")
print("Le baseline devine souvent, parfois bien, tandis que CoT décompose ... ")
print("le raisonnement étape par étape → réponse correcte = 9")
print("="*60)


📊 Problème mathématique: Comptage de moutons
Question: Un fermier possède 17 moutons. Tous sauf 9 s'enfuient. Combien en reste-t-il ?

_______________________________________________________

1°) Basique (sans guidage) :
_______________________________________________________

Si tous sauf 9 moutons s'enfuient, cela signifie qu'il reste 9 moutons. Donc, le fermier a toujours 9 moutons.


_______________________________________________________

2°) Avec chaîne de pensée ou CoT (Chain-of-Thought) :
_______________________________________________________

Pour résoudre ce problème, examinons la situation.

1. Le fermier a un total de 17 moutons.
2. La phrase "Tous sauf 9 s'enfuient" signifie que 9 moutons restent avec le fermier.
3. Cela implique que tous les autres moutons (c'est-à-dire 17 - 9 = 8 moutons) s'enfuient.

Ainsi, le nombre de moutons qui restent avec le fermier est de 9.

Donc, la réponse finale est : **9 moutons** restent.


💡 Observation:
Le baseline devine souvent, parfo

## Step 4: Question de logique

Application du CoT sur une question de comparaison logique, avec un piège

In [30]:
#q = """If John is taller than Mary, and Mary is taller than Sam, who is the shortest?"""
q = """Si John est plus grand que Marie, et que Marie est plus grande que Sam, et que Paul est plus grand que moi, qui est le plus petit ?"""

print("\n" + "="*60)
print("🧩 Question de logique : comparaison de tailles : ")
print("="*60)
print(f"Question: {q}\n")

print("\n1°) Réponse apportée de façon directe et basique :")
baseline_logic = ask(q)

print("\n2°) Réponse avec chaîne de pensée ou CoT :")
cot_logic = ask(q + "\n\nRéfléchis étape par étape avant de donner la réponse finale.")

print("\n" + "="*60)
print("💡 Observation:")
print("   - CoT force le modèle à comparer explicitement les paires")
print("     avant de conclure, réduisant les erreurs logiques.")
print("="*60)


🧩 Question de logique : comparaison de tailles : 
Question: Si John est plus grand que Marie, et que Marie est plus grande que Sam, et que Paul est plus grand que moi, qui est le plus petit ?


1°) Réponse apportée de façon directe et basique :
D'après les informations que tu as fournies :

1. John > Marie
2. Marie > Sam
3. Paul > toi

On peut conclure que Sam est le plus petit parmi John, Marie et Sam, puisque Marie est plus grande que Sam et John est plus grand que Marie. 

Cependant, pour savoir qui est le plus petit entre Sam et toi, nous n'avons pas d'information directe comparant Sam et toi. Donc, si Paul est plus grand que toi, tu pourrais être plus petit que Sam, mais ce n'est pas certain. 

En résumé, parmi les personnes mentionnées, Sam est le plus petit par rapport à John et Marie, mais on ne peut pas déterminer avec certitude si c'est lui ou toi qui es le plus petit.

2°) Réponse avec chaîne de pensée ou CoT :
Pour déterminer qui est le plus petit, analysons les informatio

## Step 5: Question de bon sens :

Test du CoT sur une tâche de raisonnement de type "bon sens"

In [31]:
#q = """Alice put her laptop in her backpack.
#Then she went to school.
# Where is the laptop most likely now?"""
q="""Alice a mis son ordinateur portable dans son sac à dos.
Puis elle est allée à l'école à pied puis en bus.
Où se trouve probablement l'ordinateur portable maintenant ?"""


print("\n" + "="*60)
print("🎒 Raisonnement de bon sens : Emplacement du portable")
print("="*60)
print(f"Question: {q}\n")

print("\n--- Basique ---")
baseline_common = ask(q)

print("\n--- With CoT ---")
cot_common = ask(q + "\n\nRéfléchis étape par étape avant de donner la réponse finale.")

print("\n" + "="*60)
print("💡 Observation :")
print("   - CoT explicite le raisonnement de bon sens, rendant")
print("     la chaîne de causalité plus claire et vérifiable.")
print("="*60)


🎒 Raisonnement de bon sens : Emplacement du portable
Question: Alice a mis son ordinateur portable dans son sac à dos.
Puis elle est allée à l'école à pied puis en bus.
Où se trouve probablement l'ordinateur portable maintenant ?


--- Basique ---
Il est probable que l'ordinateur portable d'Alice se trouve actuellement à l'école, dans son sac à dos.

--- With CoT ---
Pour déterminer où se trouve probablement l'ordinateur portable d'Alice, examinons les étapes qu'elle a suivies :

1. **Alice met son ordinateur portable dans son sac à dos** : À ce moment-là, l'ordinateur est dans le sac à dos d'Alice.

2. **Alice va à l'école à pied** : En marchant vers l'école, il est très probable qu'elle porte toujours son sac à dos avec elle, donc l'ordinateur reste dans le sac.

3. **Alice prend le bus** : En prenant le bus, il est également probable qu'elle garde son sac à dos avec elle à bord du véhicule, ce qui signifie que l'ordinateur reste toujours dans son sac.

Ainsi, en se basant sur ces é

## Step 6: Chaîne de pensée ou CoT structurée :

Guidage du modèle pour formater son raisonnement de manière structurée.

In [ ]:
prompt = """Solve: A train leaves at 3pm going 60 mph.
Another leaves at 4pm going 90 mph.
When will the faster train catch up?

Answer format:
Reasoning:
Final Answer:
"""

print("\n" + "="*60)
print("🚂 STRUCTURED CoT: Train Problem")
print("="*60)
print("Question: Problème de trains avec sortie structurée\n")

structured_answer = ask(prompt)

print("\n" + "="*60)
print("💡 Observation:")
print("Le format structuré est idéal pour les agents qui doivent")
print("parser séparément le raisonnement et la réponse finale.")
print("="*60)

In [36]:
prompt = """Résolution du problème suivant :
Un train part à 15h à 96 km/h.
Un autre part à 16h à 145 km/h.
Quand le train le plus rapide rattrapera-t-il le premier ?

Format de la réponse :
Raisonnement :
Réponse finale :
"""
print("\n" + "="*80)
print("🚂 Chaîne de pensée structurée ou STRUCTURED CoT : ")
print("=> Le fameux problème des trains")
print("="*80)
print("Question : Problème des trains avec sortie structurée\n")
print("\n"+prompt+"\n")
structured_answer = ask(prompt)

print("\n" + "="*60)
print("💡 Observation:")
print("- Le format structuré est idéal pour les agents qui doivent")
print("  parser séparément le raisonnement et la réponse finale.")
print("="*60)


🚂 Chaîne de pensée structurée ou STRUCTURED CoT : 
=> Le fameux problème des trains
Question : Problème des trains avec sortie structurée


Résolution du problème suivant : 
Un train part à 15h à 96 km/h.
Un autre part à 16h à 145 km/h.
Quand le train le plus rapide rattrapera-t-il le premier ?

Format de la réponse :
Raisonnement :
Réponse finale :


Raisonnement :
1. Le premier train part à 15h et roule à une vitesse de 96 km/h. À 16h (1 heure après le départ), il aura parcouru :  
   \( 96 \, \text{km/h} \times 1 \, \text{h} = 96 \, \text{km} \).

2. Le deuxième train part à 16h et roule à une vitesse de 145 km/h. Pour savoir quand il rattrapera le premier train, on doit déterminer quand la distance parcourue par le deuxième train sera égale à la distance parcourue par le premier train plus son avance de 96 km.

3. Soit \( t \) le temps en heures que le deuxième train roule après 16h. À ce moment-là, la distance parcourue par le premier train est :  
   \( 96 \, \text{km} + 96 \, \

## Step 7: Mini-Projet — Fonction d'évalusation :

Construction d'un évaluateur pour comparer systématiquement le raisonnement basique oie-shot (baseline) vs CoT sur plusieurs tâches.

In [39]:
# Définition des tâches de test
#tasks = [
#     "What is 23*17?",
#    "If it rains, the ground gets wet. It is raining. What can we conclude?",
#    "A red ball is in the box. The box is tipped over. Where is the ball?"
#]

tasks = [
"Combien font 124 × 18 x 0 ? ",
"S'il pleut, le sol est mouillé. Il pleut à présent. Que pouvons-nous en conclure ?",
"Une balle rouge est dans une boîte trouée grillagée de maille très fine argentée. La boîte est renversée. Où est la balle ?"
]

print("\n" + "="*70)
print("🔬 MINI-PROJET: Comparaison systématique entre la réponse **Baseline** vs **Chaîne de pensée ou CoT**")
print("="*70)

for i, t in enumerate(tasks, 1):
    print(f"\n{'─'*70}")
    print(f"📝 Task {i}/{len(tasks)}")
    print(f"{'─'*70}")
    print(f"Q: {t}\n")

    # Baseline
    print("🔹 Baseline (sans guidage):")
    baseline = ask(t, show=False)
    print(f"   {baseline.strip()}\n")

    # Chain-of-Thought
    print("🔸 Chain-of-Thought:")
    cot = ask(t + "\nThink step by step.", show=False)
    print(f"   {cot.strip()}")

print(f"\n{'='*70}")
print("✅ Évaluation terminée!")
print("="*70)


🔬 MINI-PROJET: Comparaison systématique entre la réponse **Baseline** vs **Chaîne de pensée ou CoT**

──────────────────────────────────────────────────────────────────────
📝 Task 1/3
──────────────────────────────────────────────────────────────────────
Q: Combien font 124 × 18 x 0 ? 

🔹 Baseline (sans guidage):
   Le résultat de 124 × 18 × 0 est 0, car multiplier par zéro donne toujours zéro.

🔸 Chain-of-Thought:
   Pour trouver le produit de \( 124 \times 18 \times 0 \), nous allons procéder étape par étape :

1. **Calculer \( 124 \times 18 \)** : Peu importe le résultat de ce produit, nous le multiplierons par 0 à la fin.
2. **Multipliez par 0** : En mathématiques, n'importe quel nombre multiplié par 0 donne 0.

Donc, peu importe la valeur de \( 124 \times 18 \), le produit final sera :

\[
124 \times 18 \times 0 = 0
\]

La réponse est donc \( 0 \).

──────────────────────────────────────────────────────────────────────
📝 Task 2/3
──────────────────────────────────────────────────

## 📊 Analyse et conlusion de nos différents essais :

Créons un tableau récapitulatif pour analyser les différences.

In [44]:
import pandas as pd
from IPython.display import display, HTML

# Création d'un tableau récapitulatif
summary_data = {
    'Type de tâche': [
        'Mathématiques',
        'Logique',
        'Bon sens',
        'Structuré'
    ],
    'Baseline': [
        'Réponse directe, parfois incorrecte',
        'Conclusion rapide sans justification',
        'Réponse courte',
        'Format non structuré'
    ],
    'Chain-of-Thought': [
        'Décomposition étape par étape → précision',
        'Comparaisons explicites → fiabilité',
        'Raisonnement explicite → traçabilité',
        'Format parseable → automatisation'
    ],
    'Bénéfice principal': [
        '↑ Précision',
        '↓ Erreurs logiques',
        '↑ Transparence',
        '↑ Automatisation'
    ]
}

df = pd.DataFrame(summary_data)


print("\n" + "="*70)
print("📈 TABLEAU RÉCAPITULATIF: Baseline vs Chain-of-Thought")
print("="*70 + "\n")

display(df)

print("\n" + "="*70)
print("🎯 Conclusions clés:")
print("="*70)
print("""\n1. ✅ CoT réduit les erreurs et les hallucinations
2. ✅ CoT améliore la transparence du raisonnement
3. ✅ CoT permet un parsing structuré pour les agents
4. ✅ CoT est particulièrement efficace pour:
   - Problèmes mathématiques complexes
   - Raisonnement logique multi-étapes
   - Tâches nécessitant une justification

5. 🚀 Prochaines étapes:
   - Lab 2: ReAct (Reasoning + Acting)
   - Lab 3: Planning loops
   - Lab 4: Multi-agent systems
""")
print("="*70)



📈 TABLEAU RÉCAPITULATIF: Baseline vs Chain-of-Thought



,Type de tâche,Baseline,Chain-of-Thought,Bénéfice principal
0,Mathématiques,"Réponse directe, parfois incorrecte",Décomposition étape par étape → précision,↑ Précision
1,Logique,Conclusion rapide sans justification,Comparaisons explicites → fiabilité,↓ Erreurs logiques
2,Bon sens,Réponse courte,Raisonnement explicite → traçabilité,↑ Transparence
3,Structuré,Format non structuré,Format parseable → automatisation,↑ Automatisation



🎯 Conclusions clés:

1. ✅ CoT réduit les erreurs et les hallucinations
2. ✅ CoT améliore la transparence du raisonnement
3. ✅ CoT permet un parsing structuré pour les agents
4. ✅ CoT est particulièrement efficace pour:
   - Problèmes mathématiques complexes
   - Raisonnement logique multi-étapes
   - Tâches nécessitant une justification

5. 🚀 Prochaines étapes:
   - Lab 2: ReAct (Reasoning + Acting)
   - Lab 3: Planning loops
   - Lab 4: Multi-agent systems



## 🎓 Exercices supplémentaires (Optionnel)

Pour aller plus loin, essayez ces variantes:

In [47]:
# Exercice 1: Few-shot CoT
print("\n🎯 EXERCICE 1 : Raisonnement en chaîne rapide\n")

few_shot_prompt = """Q1: Roger a 5 balles de tennis. Il achète 2 boîtes de balles de tennis supplémentaires.

Chaque boîte contient 3 balles de tennis. Combien de balles de tennis a-t-il maintenant ?

R1:

Q2: La cafétéria avait 23 pommes. S'ils en ont utilisé 20 pour préparer le déjeuner et qu'ils en ont acheté 6 de plus,

combien de pommes leur reste-t-il ?

R2:"""

print("Prompt avec exemples:")
ask(few_shot_prompt)


🎯 EXERCICE 1 : Raisonnement en chaîne rapide

Prompt avec exemples:
R1: Roger a initialement 5 balles de tennis. Il achète 2 boîtes contenant chacune 3 balles. Donc, le nombre total de balles dans les boîtes est \(2 \times 3 = 6\) balles. En ajoutant cela aux 5 balles qu'il a déjà, il a maintenant \(5 + 6 = 11\) balles de tennis.

R2: La cafétéria avait initialement 23 pommes. Ils en ont utilisé 20, donc il leur en reste \(23 - 20 = 3\) pommes. Ensuite, ils achètent 6 pommes de plus. Donc, le total final de pommes est \(3 + 6 = 9\) pommes.


"R1: Roger a initialement 5 balles de tennis. Il achète 2 boîtes contenant chacune 3 balles. Donc, le nombre total de balles dans les boîtes est \\(2 \\times 3 = 6\\) balles. En ajoutant cela aux 5 balles qu'il a déjà, il a maintenant \\(5 + 6 = 11\\) balles de tennis.\n\nR2: La cafétéria avait initialement 23 pommes. Ils en ont utilisé 20, donc il leur en reste \\(23 - 20 = 3\\) pommes. Ensuite, ils achètent 6 pommes de plus. Donc, le total final de pommes est \\(3 + 6 = 9\\) pommes."

In [48]:
# Exercice 2: Comparaison de différents modèles (si disponibles)
print("\n🎯 EXERCICE 2: Variantes de prompts CoT\n")

test_question = "Une batte de baseball et une balle coûtent 1,10€ au total. La batte coûte 1€ de plus que la balle. Combien coûte la balle ?"

variants = [
    ("Direct", test_question),
    ("CoT simple", test_question + "\nRéfléchis étape par étape."),
    ("CoT détaillée", test_question + "\nRésolvons ce problème attentivement, étape par étape, en vérifiant notre démarche."),
    ("CoT avec vérification", test_question + "\nRéfléchis étape par étape, puis vérifie que la réponse est cohérente.")
]

for variant_name, prompt in variants:
    print(f"\n--- {variant_name} ---")
    ask(prompt, show=True)
    print()


🎯 EXERCICE 2: Variantes de prompts CoT


--- Direct ---
Soit \( x \) le prix de la balle. Selon l'énoncé, la batte coûte 1€ de plus que la balle, donc le prix de la batte est \( x + 1 \).

L'équation qui représente le problème est donc :

\[
x + (x + 1) = 1,10
\]

En résolvant cette équation :

\[
2x + 1 = 1,10
\]

On soustrait 1 des deux côtés :

\[
2x = 1,10 - 1
\]
\[
2x = 0,10
\]

Ensuite, on divise par 2 :

\[
x = 0,10 / 2
\]
\[
x = 0,05
\]

Ainsi, la balle coûte 0,05€, soit 5 centimes.


--- CoT simple ---
Pour résoudre ce problème, nous allons définir les variables et procéder étape par étape.

1. **Définir les variables** :
   - Appelons le prix de la balle \( x \) euros.
   - Puisque la batte coûte 1 euro de plus que la balle, le prix de la batte sera \( x + 1 \) euros.

2. **Établir l’équation** :
   - Selon l'énoncé, le total des coûts de la batte et de la balle est de 1,10€ :
   \[
   x + (x + 1) = 1,10
   \]

3. **Simplifier l'équation** :
   - En regroupant les termes, no

## 📚 Ressources complémentaires

- **Paper original**: [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/abs/2201.11903)
- **Documentation OpenAI**: [Prompt Engineering Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- **Best Practices**: [Techniques to improve reliability](https://github.com/openai/openai-cookbook)

---

## ✅ Les TP de ce Lab sont finalisés !

Vous avez maintenant:
- ✅ Compris le principe du Chain-of-Thought prompting
- ✅ Comparé l'approche "baseline" vs "Chaîne de pensée" ou CoT sur différents types de tâches
- ✅ Expérimenté avec des formats structurés
- ✅ Construit un évaluateur systématiqu des 2 approchaes

**Prochaine étape**: LAB30 - ReAct (Reasoning + Acting) pour combiner raisonnement et actions.